# Pinecone Serverless Reranking Daily Challenge

## Part 1: Load Documents & Execute Reranking Model

### 1. Install Pinecone libraries

In [ ]:
!pip install -U pinecone-client==6.0.1 pinecone-notebooks

### 2. Authenticate with Pinecone

In [ ]:
import os
if not os.environ.get("PINECONE_API_KEY"):
    from pinecone_notebooks.colab import Authenticate
    Authenticate()

### 3. Instantiate the Pinecone client

In [ ]:
from pinecone import Pinecone
api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=api_key)

### 4. Define your query & documents

In [ ]:
query = "Tell me about Apple's products"
documents = [
    "Apple is a nutrient-rich fruit that comes in various colors like red, green, and yellow.",
    "Apple Inc. designs and manufactures consumer electronics like the iPhone, iPad, and Mac.",
    "Apples are high in fiber and vitamin C, making them a healthy snack choice.",
    "Apple recently announced its new Vision Pro headset for spatial computing.",
    "The Granny Smith apple is known for its tart flavor and crisp texture."
]

### 5. Call the reranker

In [ ]:
from pinecone import RerankModel

reranked = pc.inference.rerank(
    model="bge-reranker-v2-m3",
    query=query,
    documents=[{"id": str(i), "text": doc} for i, doc in enumerate(documents)],
    top_n=3
)

### 6. Inspect reranked results

In [ ]:
def show_reranked_results(query, matches):
    print(f"Query: {query}")
    for i, m in enumerate(matches):
        print(f"Rank: {i+1}")
        print(f"Score: {m.score}")
        print(f"Text: {m.document.text}")
        print("-" * 30)

show_reranked_results(query, reranked.data)

## Part 2: Setup a Serverless Index for Medical Notes

### 1. Install data & model libraries

In [ ]:
!pip install pandas torch transformers

### 2. Import modules & define environment settings

In [ ]:
import os
import time
import pandas as pd
from pinecone import Pinecone, ServerlessSpec
from transformers import AutoTokenizer, AutoModel
import torch

cloud = os.getenv('PINECONE_CLOUD', 'aws')
region = os.getenv('PINECONE_REGION', 'us-east-1')

spec = ServerlessSpec(cloud=cloud, region=region)

index_name = 'medical-notes-index'

### 3. Create or recreate the index

In [ ]:
if pc.has_index(name=index_name):
    pc.delete_index(name=index_name)

pc.create_index(
    name=index_name,
    dimension=384,
    metric='cosine',
    spec=spec
)

## Part 3: Load the Sample Data

### 1. Download & read JSONL

In [ ]:
import requests
import tempfile

with tempfile.TemporaryDirectory() as tmpdirname:
    file_path = os.path.join(tmpdirname, "sample_notes_data.jsonl")

    url = "https://raw.githubusercontent.com/pinecone-io/examples/refs/heads/master/docs/data/sample_notes_data.jsonl"
    response = requests.get(url)
    response.raise_for_status()

    with open(file_path, "wb") as f:
        f.write(response.content)

    df = pd.read_json(file_path, orient='records', lines=True)

### 2. Preview the DataFrame

In [ ]:
print("Data shape:", df.shape)
df.head()

## Part 4: Upsert Data into the Index

### 1. Instantiate index client & upsert

In [ ]:
index = pc.Index(name=index_name)
index.upsert_from_dataframe(df)

### 2. Wait for availability

In [ ]:
def is_fresh(index):
    stats = index.describe_index_stats()
    vector_count = stats.total_vector_count
    print(f"Vector count: ", vector_count)
    return vector_count > 0

while not is_fresh(index):
    time.sleep(5)

print("Index ready!")
index.describe_index_stats()

## Part 5: Query & Embedding Function

### 1. Define your embedding function

In [ ]:
def get_embedding(input_question):
    model_name = 'sentence-transformers/all-MiniLM-L6-v2'
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    
    encoded_input = tokenizer(input_question, padding=True, truncation=True, return_tensors='pt')
    
    with torch.no_grad():
        model_output = model(**encoded_input)
    
    embedding = model_output.last_hidden_state[0].mean(dim=0)
    return embedding

### 2. Run a semantic search query

In [ ]:
question = "What are the signs of a concussion?"
query = get_embedding(question).tolist()

results = index.query(vector=[query], top_k=5, include_metadata=True)

sorted_matches = sorted(results['matches'], key=lambda x: x['score'], reverse=True)

## Part 6: Display & Rerank Clinical Notes

### 1. Display initial search results

In [ ]:
def show_results(question, matches):
    print(f'Question: \'{question}\'')
    print('\nResults:')
    for i, match in enumerate(matches):
        print(f'{str(i+1).rjust(4)}. ID: {match["id"]}')
        print(f' Score: {match["score"]}')
        print(f' Metadata: {match["metadata"]}')
        print('')

show_results(question, sorted_matches)

### 2. Prepare documents for reranking

In [ ]:
transformed_documents = [
    {
        'id': match['id'],
        'reranking_field': '; '.join([f"{key}: {value}" for key, value in match['metadata'].items()])
    }
    for match in results['matches']
]

### 3. Execute serverless reranking

In [ ]:
refined_query = "What are the immediate symptoms of a head injury like a concussion?"

reranked_results = pc.inference.rerank(
    model="bge-reranker-v2-m3",
    query=refined_query,
    documents=transformed_documents,
    rank_fields=["reranking_field"],
    top_n=3,
    return_documents=True,
)

### 4. Show reranked results

In [ ]:
def show_reranked_results_final(question, matches):
    print(f'Question: \'{question}\'')
    print('\nReranked Results:')
    for i, match in enumerate(matches):
        print(f'{str(i+1).rjust(4)}. ID: {match.document.id}')
        print(f' Score: {match.score}')
        print(f' Reranking Field: {match.document.reranking_field}')
        print('')
        
show_reranked_results_final(refined_query, reranked_results.data)

### 5. Clean up

In [ ]:
pc.delete_index(name=index_name)